In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('default')

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': False, 'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters, 'refit_full': False,
                'set_best_to_refit_full': False, 'save_bag_folds': False}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(train_data, presets=custom_preset, excluded_model_types=['KNN'])  

No path specified. Models will be saved in: "AutogluonModels\ag-20240821_072221"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       21.61 GB / 31.75 GB (68.1%)
Disk Space Avail:   832.49 GB / 952.44 GB (87.4%)
Presets specified: [{'auto_stack': False, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': {}, 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'], 'CAT': {}, 'XGB': {}, 'FASTAI': {}, 'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}], 'XT': [{'criterion': 'gin

In [ ]:
# %%

metrics = ['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test', 'fit_time']

df_leaders = predictor.leaderboard(test_data)

df_leaders.head(40)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L2,-0.141744,-0.128997,log_loss,0.384685,127.157301
1,CatBoost,-0.158654,-0.150310,log_loss,0.033089,89.332944
2,XGBoost,-0.171801,-0.144754,log_loss,0.184851,4.192184
3,NeuralNetFastAI,-0.172637,-0.168564,log_loss,0.089002,27.519064
4,LightGBM,-0.184182,-0.161321,log_loss,0.064970,2.486282
5,NeuralNetTorch,-0.186882,-0.163288,log_loss,0.031303,92.630702
6,LightGBMXT,-0.193761,-0.178918,log_loss,0.202416,5.422224
7,RandomForestEntr,-0.196384,-0.194874,log_loss,0.168260,1.104101
8,RandomForestGini,-0.208654,-0.206937,log_loss,0.188910,0.998365
9,ExtraTreesEntr,-0.260219,-0.267746,log_loss,0.234466,0.822240


In [ ]:
# %%

df_leaders.tail(20)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L2,-0.141744,-0.128997,log_loss,0.384685,127.157301
1,CatBoost,-0.158654,-0.150310,log_loss,0.033089,89.332944
2,XGBoost,-0.171801,-0.144754,log_loss,0.184851,4.192184
3,NeuralNetFastAI,-0.172637,-0.168564,log_loss,0.089002,27.519064
4,LightGBM,-0.184182,-0.161321,log_loss,0.064970,2.486282
5,NeuralNetTorch,-0.186882,-0.163288,log_loss,0.031303,92.630702
6,LightGBMXT,-0.193761,-0.178918,log_loss,0.202416,5.422224
7,RandomForestEntr,-0.196384,-0.194874,log_loss,0.168260,1.104101
8,RandomForestGini,-0.208654,-0.206937,log_loss,0.188910,0.998365
9,ExtraTreesEntr,-0.260219,-0.267746,log_loss,0.234466,0.822240


In [ ]:
# %%

y_pred = predictor.predict_proba(test_data.drop(columns=[label]))
y_pred.head()

# predictor.evaluate(test_data, silent=True)

,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
0,0.000017,7.858799e-06,1.361795e-05,1.543366e-04,0.999590,0.000157,0.000027,0.000007,0.000007,0.000007,6.145465e-06,2.306060e-06,0.000003
1,0.000033,1.871548e-05,1.488291e-05,1.593300e-05,0.000180,0.612613,0.386925,0.000087,0.000041,0.000018,1.779206e-05,1.423204e-05,0.000021
2,0.000002,8.209640e-07,6.017985e-07,8.544085e-07,0.000001,0.000288,0.999697,0.000005,0.000001,0.000001,6.827307e-07,7.310827e-07,0.000001
3,0.000002,1.066635e-06,1.051127e-06,3.235323e-05,0.000021,0.000002,0.000005,0.000085,0.998073,0.001768,7.033942e-06,1.018653e-06,0.000003
4,0.000003,3.862884e-06,1.251074e-06,1.021575e-06,0.000001,0.000003,0.039631,0.960230,0.000118,0.000002,7.882075e-07,1.443007e-06,0.000003
